**DIGITAL** **WATERMARKING USING SPATIAL DOMAIN METHOD**

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Fungsi untuk menyisipkan watermark ke dalam gambar menggunakan LSB
def insert_watermark(image, watermark, bit_position=0):
    # Mengubah watermark menjadi gambar biner
    watermark = cv2.resize(watermark, (image.shape[1], image.shape[0]))  # Sesuaikan ukuran watermark dengan gambar
    if len(watermark.shape) == 3:  # Jika gambar watermark masih berwarna (RGB)
        watermark = watermark[:, :, 0]  # Ambil hanya kanal pertama (grayscale)

    # Menyisipkan watermark ke dalam bit terendah dari gambar
    watermark_bin = np.unpackbits(watermark.astype(np.uint8), axis=None)  # Mengubah watermark ke dalam bentuk biner
    image_flat = image.flatten()  # Meratakan gambar ke dalam vektor
    image_flat_with_watermark = image_flat.copy()

    for i in range(len(watermark_bin)):
        image_flat_with_watermark[i] = (image_flat_with_watermark[i] & 0xFE) | watermark_bin[i]

    # Mengembalikan gambar dengan watermark
    watermarked_image = np.reshape(image_flat_with_watermark, image.shape)
    return watermarked_image

The insert_watermark function is designed to embed a watermark into an image using the Least Significant Bit (LSB) method. The process begins by resizing the watermark to match the dimensions of the main image using the cv2.resize function. If the watermark is a colored (RGB) image, only the first channel is used to simplify the process. Next, the watermark is converted into a binary format using np.unpackbits, where each pixel is represented as a sequence of bits. The main image is then flattened into a one-dimensional vector for easier data manipulation. The watermark embedding is performed by modifying the least significant bit (LSB) of each pixel in the image using bitwise operations: the LSB is cleared (& 0xFE) and replaced with the corresponding watermark bit (| watermark_bin[i]). Once all the watermark bits are embedded, the modified image vector is reshaped back to its original dimensions using np.reshape. The final result is a new image with the watermark hidden in the least significant bits of each pixel.

In [ ]:
# Fungsi untuk mengekstrak watermark dari gambar yang sudah di-watermark
def extract_watermark(watermarked_image, original_image):
    # Meratakan kedua gambar
    original_image_flat = original_image.flatten()
    watermarked_image_flat = watermarked_image.flatten()

    # Mengekstrak watermark dengan cara membandingkan bit terendah
    extracted_watermark = []
    for i in range(len(original_image_flat)):
        extracted_bit = watermarked_image_flat[i] & 1  # Mengambil bit terendah
        extracted_watermark.append(extracted_bit)

    # Mengonversi hasil ekstraksi biner menjadi gambar
    extracted_watermark = np.array(extracted_watermark).reshape(original_image.shape)
    return extracted_watermark

The extract_watermark function is designed to retrieve the hidden watermark from an image that has been embedded using the Least Significant Bit (LSB) method. The process starts by flattening both the original image and the watermarked image into one-dimensional arrays using the flatten() method. This allows for a direct comparison of pixel data between the two images. To extract the watermark, the function iterates over each pixel and isolates the least significant bit (LSB) of the watermarked image using the bitwise operation & 1, which extracts only the lowest bit of the pixel value. These extracted bits are stored in a list that represents the binary form of the watermark. Once all bits are extracted, they are reshaped to match the dimensions of the original image using np.reshape, converting the binary data back into a visual format.

In [ ]:
from google.colab import files
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Unggah file gambar
print("Upload gambar asli:")
uploaded = files.upload()
image_path = list(uploaded.keys())[0]

print("Upload watermark:")
uploaded = files.upload()
watermark_path = list(uploaded.keys())[0]

# Membaca gambar utama dalam format berwarna (BGR)
image = cv2.imread(image_path, cv2.IMREAD_COLOR)

# Membaca watermark dalam grayscale
watermark = cv2.imread(watermark_path, cv2.IMREAD_GRAYSCALE)

# Sesuaikan ukuran watermark agar sesuai dengan gambar utama
watermark_resized = cv2.resize(watermark, (image.shape[1], image.shape[0]))

# Konversi watermark grayscale ke format BGR
watermark_bgr = cv2.cvtColor(watermark_resized, cv2.COLOR_GRAY2BGR)

# Terapkan watermark (blending)
alpha = 0.3  # Transparansi watermark
beta = 1 - alpha
gamma = 0  # Offset
watermarked_image = cv2.addWeighted(image, beta, watermark_bgr, alpha, gamma)

# Simpan gambar hasil berwarna dengan watermark
output_path = "watermarked_image_color.png"
cv2.imwrite(output_path, watermarked_image)
print(f"Gambar dengan watermark disimpan di: {output_path}")

# Tampilkan hasil menggunakan matplotlib
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.title("Original Image")
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.axis("off")

plt.subplot(1, 3, 2)
plt.title("Watermark (Resized)")
plt.imshow(watermark_resized, cmap="gray")
plt.axis("off")

plt.subplot(1, 3, 3)
plt.title("Watermarked Image")
plt.imshow(cv2.cvtColor(watermarked_image, cv2.COLOR_BGR2RGB))
plt.axis("off")

plt.show()